## Step 1: Setup and Imports

We begin by setting the working directory and importing the custom processing functions defined in `SignalProcessingScripts.py`.

We also reload the module to ensure any changes to the code are reflected without restarting the kernel.

The `context` dictionary is used to store and pass data between steps.

In [ ]:
%cd "/home/ammir/Desktop/Golem Seismic Processing/src/"
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # for more resolution plots

# Center the plot
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

import golem as gisis

context = {}

## 2. Read Binary Header
    First we will read the binary header of the segy file.

In [ ]:
context['binary_header'] = gisis.get_binary_header(context, file_path='/home/ammir/Desktop/Seismic Dataset/BP Benchmark 2004/ShotGather/shots0001_0200.segy')
context['binary_header']

## 3. Read Text Header
    Then we obtain the text header to see informations about the geometry or past processing.

In [ ]:
context['text_header'] = gisis.get_text_header(context, file_path='/home/ammir/Desktop/Seismic Dataset/BP Benchmark 2004/ShotGather/shots0001_0200.segy')
context['text_header']

## 4. Read Trace Data
    Here we extract the trace data, which contains the seismogram in a 2D numpy array with dimensions (timesamples, ntraces)

In [ ]:
context['data'] = gisis.get_trace_data(context, file_path='/home/ammir/Desktop/Seismic Dataset/BP Benchmark 2004/ShotGather/shots0001_0200.segy', ignore_geometry=True)

## 5. Read Trace Header (trace geometry)
    Here we extract the trace header, which contains the geometry information about the segy file

In [ ]:
context['geometry'] = gisis.get_trace_header(context, file_path='/home/ammir/Desktop/Seismic Dataset/BP Benchmark 2004/ShotGather/shots0001_0200.segy',ignore_geometry=True)
context['geometry'].head(10)

## 6. Scale coordinate units
    Here we need to scale the coordinate units of the segy. This happens because the file can only store in integers, but X, Y and Z are in floats, so we need to apply a scaling factor to make it a float

In [ ]:
context['geometry'] = gisis.scale_coordinate_units(context, key="geometry", XY_headers=['SourceX', 'SourceY', 'GroupX', 'GroupY'], elevation_headers=['SourceDepth', 'SourceSurfaceElevation'], XY_Scaler=10., elevation_scaler=10.)
context['geometry'].head(10)

## 7. Subset a specific shot for visualization
    Let's subset the data into the shot number 180 just to see its contents

In [ ]:
gisis.subset_geometry_by_condition(context, condition="FieldRecord == 180", key_input="data", key_output="subset_data", key_geometry_input="geometry", key_geometry_output="subset_geometry")

## 8. Visualize the data

In [ ]:
gisis.plot_seismic_image(context,
    xlabel="Receiver X (m)",
    ylabel="Time (ms)",
    y_spacing=6.0,
    x_header="GroupX",
    perc=95,
    key_data='subset_data',
    key_geometry="subset_geometry",
    figure_dims=(20,12)
)

## 9. Plot the acquisition geometry

In [ ]:
gisis.plot_acquisition(context, key_acquisition_geometry='geometry')

## 10. Read Velocity Model SEGY
    Now let's do the same for the velocity model

In [ ]:
model_path = '/home/ammir/Desktop/Seismic Dataset/BP Benchmark 2004/Properties/vel_z6.25m_x12.5m_exact.segy'
context['model_text_header'] = gisis.get_text_header(context, file_path=model_path)
context['model_text_header']

## 11. Read Geometry 

In [ ]:
context['model geometry'] = gisis.get_trace_header(context, file_path=model_path,ignore_geometry=True)
context['model geometry'].head(10)

## 11. Scale Coordinate Units

In [ ]:
context['model geometry'] = gisis.scale_coordinate_units(context, key="model geometry", XY_headers=['SourceX', 'SourceY', 'GroupX', 'GroupY'], elevation_headers=['SourceDepth', 'SourceSurfaceElevation'], XY_Scaler=10., elevation_scaler=10.)
context['model geometry'].head(10)

## 12. Read Velocity Model Contents

In [ ]:
context['model'] = gisis.get_trace_data(context, file_path=model_path, ignore_geometry=True)

## 13. Show Contents

In [ ]:
gisis.plot_seismic_image(context,
    xlabel="X (m)",
    ylabel="Depth (m)",
    y_spacing=6.25,
    x_header="GroupX",
    key_data='model',
    key_geometry="model geometry",
    figure_dims=(20,12),
    cmap='jet'
)

# 14. Plot The Acquisition
    Let's plot the acquisition but now adding the velocity model as boundary for the polygon

In [ ]:
gisis.plot_acquisition(context, key_acquisition_geometry='geometry', key_model_geometry='model geometry')

## 15. Convert Coordinates to a Local System
- Retrieve the model geometry DataFrame from the context.
- Define corner coordinates using the minimum and maximum values of `SourceX` and `SourceY`.
- Calculate the azimuth angle using the `calculate_azimuth` function.
- Convert the acquisition coordinates (e.g., `SourceX`, `SourceY`) to local coordinates using the `get_local_coordinates` function.
- Update the geometry DataFrame in the context with these local coordinates.

In [ ]:
gisis.generate_local_coordinates(context, key_geometry='geometry',key_model_geometry='model geometry')

# 16. Plot The Acquisition
    Let's plot the acquisition but now with local geometry defined by velocity model

In [ ]:
gisis.plot_acquisition(context, key_acquisition_geometry='geometry', key_model_geometry='model geometry')

# 17. Kill Traces Outside Model
    Since we have traces that are outside the model, which were modeled by extending the model to the left, we want to get rid of them and stay with only the traces that are inside

In [ ]:
gisis.kill_traces_outside_box(context, key_geometry='geometry',key_data='data')

# 18. Plot Result Geometry
    Now let's plot it to see how our acquisition looks like

In [ ]:
gisis.plot_acquisition(context, key_acquisition_geometry='geometry', key_model_geometry='model geometry')

## 19. Save Processed Geometry Data
- Define a function to store the geometry DataFrame as a Parquet file:
  - Validate that the context and the geometry DataFrame exist.
  - Check that the provided file path is valid and the destination directory exists.
  - Write the DataFrame to a Parquet file using the `to_parquet` method.

In [ ]:
gisis.store_geometry_as_parquet(context, file_path='SourceReceiverGeometry.parquet',key_geometry='geometry')
gisis.store_geometry_as_parquet(context, file_path='ModelGeometry.parquet',key_geometry='model geometry')

# 20. Writing Seismogram

In [ ]:
gisis.write_data(context,"./", "SeismogramKillTraces.npy", format='npy')